<a href="https://colab.research.google.com/github/lucastanger/vehicle_classification/blob/main/vehicle_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evidencetheory / Dempsters Rule

In [110]:
!pip install py_dempster_shafer

In [111]:
# Imports
import pandas as pd
from pyds import MassFunction

# Settings

In [112]:
#@title Define classification bounds
# 
SPEED_LOWER_BOUND = 60 #@param {type:"integer"}
SPEED_UPPER_BOUND = 90 #@param {type:"integer"}
#@markdown ---
REFLECT_LOWER_BOUND = 0.1 #@param {type:"number"}
REFLECT_UPPER_BOUND = 0.6 #@param {type:"number"}
#@markdown ---
CAR_WIDTH_LOWER_BOUND = 1.6 #@param {type:"number"}
CAR_WIDTH_UPPER_BOUND = 2.15 #@param {type:"number"}
#@markdown ---
TRUCK_WIDTH_LOWER_BOUND = 1.9 #@param {type:"number"}
TRUCK_WIDTH_UPPER_BOUND = 2.5 #@param {type:"number"}
#@markdown ---
TWOWHEEL_WIDTH_LOWER_BOUND = 0.7 #@param {type:"number"}
TWOWHEEL_WIDTH_UPPER_BOUND = 1.1 #@param {type:"number"}
#@markdown ---
DISTANCE_PERCENTAGE_THRESHOLD = 1.05 #@param {type:"number"}

# Load data

We received 6 datasets, containing different radar measurements. The data consists of the following features:

* Takt (*Clock*) - A 100ms interval
* Radar/Distanz (*Radar/Distance in meters*) - The distance to the object in front
* Breite (*Width in meters*) - The width of the object in front
* Reflektionsstärke (*Reflection strength*) - The measured reflection strength of the object in front
* Geschwindigkeit (*Speed/Velocity*) - The measured velocity of the object in front

In [113]:
dfs = []

# Load data iteratively
for i in range(1, 7):
  # Store dataframes in list for later access
  dfs.append(pd.read_csv('https://raw.githubusercontent.com/lucastanger/vehicle_classification/main/data/DE11_{}.csv'.format(i), sep=";"))

# Print out the head of the dataframes
for df in dfs: print(df.head(3))

   Takt Radar / Distanz (m) Breite (m) Reflektionsstaerke Geschwindigkeit
0     1                61,2      0,735           0,050715          119,56
1     2                60,8       0,63         0,03803625          119,92
2     3                60,5       0,42           0,050715          120,64
   Takt Radar / Distanz (m) Breite (m) Reflektionsstaerke Geschwindigkeit
0     1                  75       2,52              0,432           81,36
1     2                75,1        2,4             0,4032           81,36
2     3                75,2       2,52               0,36              81
   Takt Radar / Distanz (m) Breite (m) Reflektionsstaerke Geschwindigkeit
0     1                  60        1,1              0,072            90,5
1     2                  58        1,2              0,069            82,8
2     3                  56       1,21              0,073            82,8
   Takt Radar / Distanz (m) Breite (m) Reflektionsstaerke Geschwindigkeit
0     1                  40       0,86

In [114]:
# Define omega
# p = PKW
# l = LKW
# m = Motorrad
# f = Fahrrad
omega = 'plmf'

Vorliegende Daten:

* **Geschwindigkeit** - Mittlere Geschwindigkeiten von 60-90
km/h sprechen für PKW, LKW oder
Motorrad. Höhere Geschwindigkeiten
über 90 km/h sprechen für PKW oder
Motorrad. 

* **Distanz** - Distanz zum vorausfahrenden Objekt - Sehr schnell schwankende Änderungen der Distanz (bei gleichbleibender
Eigengeschwindigkeit, d.h. Beschleunigungen), weisen auf ein Motorrad
hin, schnelle auf einen PKW oder
Motorrad.

* **Breite (m)** - Breiten zwischen 1,60 und 2,15
weisen auf PKW hin, Breiten zwischen 1,90 und 2,50 auf einen LKW
und Breiten zwischen 0,70 und 1,10
auf ein Motorrad oder ein Fahrrad.

* **Reflektionsstaerke** - LKW und PKW liefern eine große
Reflektionsstärke. Motorräder und
Fahrräder liefern deutlich geringere
Reflektionsstärken.



In [115]:
def speed(speed: float) -> str: 
  """Classify a given speed. Limits are defined by lower and upper bounds"""
  if speed <= SPEED_LOWER_BOUND:
    return 'f'
  elif SPEED_LOWER_BOUND < speed < SPEED_UPPER_BOUND:
    return 'plm'
  elif speed >= SPEED_UPPER_BOUND:
    return 'pm'

def reflective_strength(reflective_strength: float) -> str:
  """Classify a given reflective strength. Limits are defined by lower and upper bounds"""
  if reflective_strength < REFLECT_LOWER_BOUND:
    return 'fm'
  elif REFLECT_LOWER_BOUND >= reflective_strength > REFLECT_UPPER_BOUND:
    return 'pl'
  elif reflective_strength > REFLECT_UPPER_BOUND:
    return 'l'

def width(width: float) -> str:
  """Classify a given width. Limits are definied by lower and upper bounds"""
  if CAR_WIDTH_LOWER_BOUND <= width <= CAR_WIDTH_UPPER_BOUND:
    return 'p'
  elif TRUCK_WIDTH_LOWER_BOUND <= width <= TRUCK_WIDTH_UPPER_BOUND:
    return 'l'
  elif TWOWHEEL_WIDTH_LOWER_BOUND <= width <= TWOWHEEL_WIDTH_UPPER_BOUND:
    return 'fm'
  else:
    # Return nothing if measurement is meaningless
    return ''

def distance(distance: float, old_distance: float) -> str:
  """Classify a given distance by its predecessor"""
  # Check if ahead driving object gained x% of distance within 100ms
  if distance > old_distance*DISTANCE_PERCENTAGE_THRESHOLD:
    return 'm'
  else:
    return ''


Laut Aufgabenbeschreibung funktionieren die Sensoren im Nahbereich gut und werden auf Entfernung immer schlechter. 

**Annahme:** Die Evidenz steigt in Abhängigkeit der Distanz:

Evidenz = 1 - Distanz/2 /100

Omega = 1 - Evidenz



In [116]:
def compute_evidence(distance: float) -> float:
  return 1-distance/2/100

In [117]:
def createMassFunction(om: list, distance: float) -> MassFunction:
  """ Wrapper function for creation of pyds MassFunction """
  # Remove empty elements
  om = list(filter(None, om)) 
  # Remove duplicates
  om = list(dict.fromkeys(om))
  # Create param list
  param = {}

  for elem in om:
    param[elem] = compute_evidence(distance=distance)/len(om)

  param[omega] = 1 - compute_evidence(distance=distance)

  m = MassFunction(param)

  return m

In [118]:
#@title Select dataset
old_dist = 0

mass_functions = []

dataset_number = 4 #@param ["0", "1", "2", "3", "4", "5"] {type:"raw"}

for row in dfs[dataset_number].iloc:
  omega_ = []
  for i in range(1,5):
    s = row[i]
    s = s.replace(',', '.')
    if i == 1:
      if old_dist == 0:
        omega_.append(distance(float(s), float(s)))
        old_dist = float(s)
      else:
        omega_.append(distance(float(s), old_dist))
        old_dist = float(s)
    elif i == 2:
      omega_.append(width(float(s)))
    elif i == 3:
      omega_.append(reflective_strength(float(s)))
    elif i == 4:
      omega_.append(speed(float(s)))

  print(f'Takt {row[0]}: {omega_}')
  mass_functions.append(createMassFunction(omega_, float(row[1].replace(',', '.'))))

Takt 1: ['', '', 'fm', 'f']
Takt 2: ['', 'fm', 'fm', 'f']
Takt 3: ['', '', 'fm', 'f']
Takt 4: ['', 'fm', 'fm', 'f']
Takt 5: ['', 'fm', 'fm', 'f']
Takt 6: ['', '', 'fm', 'f']
Takt 7: ['', 'fm', 'fm', 'f']
Takt 8: ['', '', 'fm', 'f']
Takt 9: ['', 'fm', 'fm', 'f']
Takt 10: ['', 'fm', 'fm', 'f']
Takt 11: ['', '', 'fm', 'f']
Takt 12: ['', '', 'fm', 'f']
Takt 13: ['', 'fm', 'fm', 'f']
Takt 14: ['', 'fm', 'fm', 'f']
Takt 15: ['', 'fm', 'fm', 'f']
Takt 16: ['', 'fm', 'fm', 'f']
Takt 17: ['', 'fm', 'fm', 'f']
Takt 18: ['', 'fm', 'fm', 'f']


In [119]:
c = None

for i, func in enumerate(mass_functions): 
  if i == 0:
    c = func
  else:
    c = c.combine_conjunctive(mass_function=mass_functions[i-1])
  print(f'{c}')

print(f'Final Massfunction: {c}')

{{'m', 'p', 'f', 'l'}:0.35; {'m', 'f'}:0.325; {'f'}:0.325}
{{'f'}:0.544375; {'m', 'f'}:0.333125; {'m', 'p', 'f', 'l'}:0.12249999999999998}
{{'f'}:0.7027046875; {'m', 'f'}:0.25993281249999994; {'m', 'p', 'f', 'l'}:0.03736249999999998}
{{'f'}:0.8119607148437501; {'m', 'f'}:0.17813822265624998; {'m', 'p', 'f', 'l'}:0.009901062499999997}
{{'f'}:0.884825937841797; {'m', 'f'}:0.1129463230957031; {'m', 'p', 'f', 'l'}:0.002227739062499999}
{{'f'}:0.9314714330158692; {'m', 'f'}:0.06810529656225583; {'m', 'p', 'f', 'l'}:0.0004232704218749997}
{{'f'}:0.9602534311492041; {'m', 'f'}:0.03967884558329588; {'m', 'p', 'f', 'l'}:6.772326749999996e-05}
{{'f'}:0.9774935053882368; {'m', 'f'}:0.022497521278819415; {'m', 'p', 'f', 'l'}:8.973332943749998e-06}
{{'f'}:0.9875088954904715; {'m', 'f'}:0.012490117442904746; {'m', 'p', 'f', 'l'}:9.870666238124998e-07}
{{'f'}:0.9931923480423068; {'m', 'f'}:0.006807563121696919; {'m', 'p', 'f', 'l'}:8.883599614312494e-08}
{{'f'}:0.9963323775077929; {'m', 'f'}:0.003667

In [120]:
def compute_plausibility(mass_function: MassFunction) -> None:
  for elem in omega:
    print(f'Plausibility of {elem}: {mass_function.pl({elem})}')

def compute_belief(mass_function: MassFunction) -> None:
  for elem in omega:
    print(f'B({elem}): {mass_function.bel({elem})}')

  print(f'B(four wheels) = B(p,l) = {c.bel("pl")}')
  print(f'B(two wheels) = B(f,m) = {c.bel("fm")}')

def compute_doubt(mass_function: MassFunction, lbl: str) -> None: 
  print(f'Doubt on classification of {lbl}: {mass_function.bel(omega.replace(lbl, ""))}')

In [121]:
compute_plausibility(c)

Plausibility of p: 1.4246383361436763e-17
Plausibility of l: 1.4246383361436763e-17
Plausibility of m: 4.247172264339871e-05
Plausibility of f: 1.0


In [122]:
compute_belief(c)

B(p): 0.0
B(l): 0.0
B(m): 0.0
B(f): 0.9999575282773566
B(four wheels) = B(p,l) = 0.0
B(two wheels) = B(f,m) = 1.0


In [129]:
#@title Compute Doubt for given label
doubt = "m" #@param ["p", "l", "m", "f"]

compute_doubt(c, doubt)

Doubt on classification of m: 0.9999575282773566


In [124]:
for i, fun in enumerate(mass_functions):
  print(f'MassFuntion at {i*100}ms: {fun}')

MassFuntion at 0ms: {{'m', 'p', 'f', 'l'}:0.35; {'m', 'f'}:0.325; {'f'}:0.325}
MassFuntion at 100ms: {{'m', 'f'}:0.34750000000000003; {'f'}:0.34750000000000003; {'m', 'p', 'f', 'l'}:0.30499999999999994}
MassFuntion at 200ms: {{'m', 'f'}:0.3675; {'f'}:0.3675; {'m', 'p', 'f', 'l'}:0.265}
MassFuntion at 300ms: {{'m', 'f'}:0.3875; {'f'}:0.3875; {'m', 'p', 'f', 'l'}:0.22499999999999998}
MassFuntion at 400ms: {{'m', 'f'}:0.405; {'f'}:0.405; {'m', 'p', 'f', 'l'}:0.18999999999999995}
MassFuntion at 500ms: {{'m', 'f'}:0.42; {'f'}:0.42; {'m', 'p', 'f', 'l'}:0.16000000000000003}
MassFuntion at 600ms: {{'m', 'f'}:0.43374999999999997; {'f'}:0.43374999999999997; {'m', 'p', 'f', 'l'}:0.13250000000000006}
MassFuntion at 700ms: {{'m', 'f'}:0.445; {'f'}:0.445; {'m', 'p', 'f', 'l'}:0.10999999999999999}
MassFuntion at 800ms: {{'m', 'f'}:0.455; {'f'}:0.455; {'m', 'p', 'f', 'l'}:0.08999999999999997}
MassFuntion at 900ms: {{'m', 'f'}:0.46125; {'f'}:0.46125; {'m', 'p', 'f', 'l'}:0.07750000000000001}
MassFunti